# Concrete Crack Prediction using CNN

This notebook implements a Convolutional Neural Network (CNN) to classify images of concrete as either "Negative" (no crack) or "Positive" (crack).

## Dataset
The dataset consists of two folders:
- `Negative`: Images without cracks.
- `Positive`: Images with cracks.

## Goal
Build a binary classification model using PyTorch and train it using GPU acceleration.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)

## 1. Device Configuration
Check if a GPU is available and set the device accordingly.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Data Preparation
Define transformations (resize, tensor conversion, normalization) and load the dataset.

In [ ]:
# Hyperparameters
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
LEARNING_RATE = 0.001
EPOCHS = 10

# Define transforms
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet stats
])

# Load dataset
dataset_path = "." # Current directory contains Negative and Positive folders
# We need to point to the parent directory of Negative/Positive if we use ImageFolder directly on them,
# but ImageFolder expects a root dir containing class folders. 
# Assuming the current directory is 'd:/PYTHON/CONCRETE_CRACK_PREDECTION_MODEL' and it has 'Negative' and 'Positive' folders.
# However, ImageFolder requires a root directory that contains the class directories.
# Since we are inside the folder that contains the classes, we might need to wrap them or just use the current dir if it works, 
# but usually it's root/class_a, root/class_b. 
# Let's assume the script runs from 'd:/PYTHON/CONCRETE_CRACK_PREDECTION_MODEL'.

full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

print(f"Classes found: {full_dataset.classes}")
print(f"Total images: {len(full_dataset)}")

# Split into Train (80%) and Validation (20%)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

## 3. Data Visualization
Visualize a batch of images to verify loading.

In [ ]:
def imshow(img, title):
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')

# Get a batch of training data
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show images
plt.figure(figsize=(12, 6))
grid_img = torchvision.utils.make_grid(images[:8], nrow=4)
imshow(grid_img, title=[full_dataset.classes[x] for x in labels[:8]])
plt.show()

## 4. Model Architecture
Define a simple CNN model.

In [ ]:
class ConcreteCNN(nn.Module):
    def __init__(self):
        super(ConcreteCNN, self).__init__()
        self.features = nn.Sequential(
            # Conv Layer 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Layer 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Layer 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512), # 224 / 2 / 2 / 2 = 28
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2) # 2 classes: Negative, Positive
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = ConcreteCNN().to(device)
print(model)

## 5. Training Setup
Define Loss function and Optimizer.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## 6. Training Loop
Train the model and evaluate on the validation set.

In [ ]:
train_losses = []
val_accuracies = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    val_accuracies.append(accuracy)
    
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}, Val Accuracy: {accuracy:.2f}%")

## 7. Evaluation Results
Plot the training loss and validation accuracy.

In [ ]:
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(val_accuracies, label='Validation Accuracy', color='orange')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()

## 8. Save Model
Save the trained model state dictionary.

In [ ]:
torch.save(model.state_dict(), 'concrete_crack_cnn.pth')
print("Model saved to concrete_crack_cnn.pth")